# Introduction

Author: Megan L Smith

Date: May 4, 2024

Purpose: Woodshole MOLE Course

This jupyter notebook will illustrate an application of machine learning in population genetics. Specifically, we will simulate data under two simple demographic models using msprime. Then, we will calculate Site Frequency Spectra (SFS) from these data, and store them as numpy matrices. We will use these matrices to train a convolutional neural network (CNN) using tensorflow and keras. Finally, we will generate an independent test dataset and test the classifier. This is a simple example that does not require machine learning, but it should illustrate the basic principles of using machine learning in python to analyze genomic data.

In [1]:
# Import python packages
# python version: 3.10.8
import msprime # version 1.2.0
import numpy as np # version 1.23.5
from scipy.spatial.distance import euclidean # version 1.9.3
import random

# sklearn imports version 1.2.0
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, accuracy_score 

# tensorflow imports version 2.10.0
from tensorflow.keras import layers, optimizers, models 

# keras imports version 2.10.0
from keras.utils import to_categorical


2024-05-06 20:37:31.828382: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Simulating genetic data in msprime

There are many simulators we could use to simulate data, and different problems require different simulators. For example, when we want to simulate selection, SLiM (Haller and Messer, 2022) may be a good choice. In this simple example, we will use msprime (Baumdicker et al., 2022). msprime integrates well into python pipelines, since it is a python package. It is also used in stdpopsim, a library for standardized population genetic simulations for many species (Adrion et al., 2020). We will keep it simple for the point of illustration and simulate data under two models: a divergence-only model and a divergence-with-gene-flow model. Our question is: do these two sister species have a history of gene flow?

<img src="Models-01.png" width="500"/>


## Specify demographic models

Below, we specify our models. We could make these models much more complex. See the msprime documentation for more examples and possibilities: https://tskit.dev/msprime/docs/stable/demography.html. 

In [2]:
# Define our models in msprime

def divergence(tdiv, ne_anc, ne_1, ne_2):  
    """This function defines a divergence-only model. 
        It takes as input several parameters:
        tdiv (divergence time in generations)
        ne_anc (the effective population size of the ancestral population)
        ne_1 (the effective population size of population 1)
        ne_2 (the effective population size of population 2)"""

     # set up msprime model
    demography = msprime.Demography()
    demography.add_population(name="A", initial_size = ne_1) # population 1
    demography.add_population(name="B", initial_size = ne_2) # population 2
    demography.add_population(name="C", initial_size = ne_anc) # ancestral population
    demography.add_population_split(time=tdiv, derived=["A", "B"], ancestral="C") # divergence
    return(demography)

def geneflow(tdiv, migrate, ne_anc, ne_1, ne_2): # UPDATED
    """This function defines a secondary contact model. 
        It takes as input several parameters:
        tdiv (divergence time in generations)
        migrate (the rate of migration between populations)
        ne_anc (the effective population size of the ancestral population)
        ne_1 (the effective population size of population 1)
        ne_2 (the effective population size of population 2)"""

     # set up msprime model
    demography = msprime.Demography()
    demography.add_population(name="A", initial_size = ne_1) # population 1
    demography.add_population(name="B", initial_size = ne_2) # population 2
    demography.add_population(name="C", initial_size = ne_anc) # ancestral population
    demography.set_symmetric_migration_rate(populations=["A", "B"], rate=migrate) # start migration
    demography.add_symmetric_migration_rate_change(time=tdiv//2, populations=["A", "B"], rate=0) # set migration rate to zero
    demography.add_population_split(time=tdiv, derived=["A", "B"], ancestral="C") # divergence (migration stops)
    return(demography)



## Specify priors and other values needed for simulations

We will specify priors for parameters to use when generating training data. We also need to define a mutation rate. We could also draw this value from a prior distribution, but for the sake of simplicity, we will set μ to a fixed value here. For this example (primarily for the sake of time) we assume there is no recombination within loci. Additionally, we need to specify the length of loci to simulate, the number of loci to simulate per replicate, the number of replicates to simulate per model, the number of SNPs to keep, and the number of individuals to sample per population. For this particular architecture, we assume equal numbers of samples from each population, but this is easily relaxed.

In [3]:
tdiv_prior=[50000, 500000] # prior on divergence time
ne_prior=[10000, 100000] # prior on ne
migrate_prior=[1e-6,1e-4] # prior on migration rate
mu = 1e-7 # mutation rate
rec_rate = 1e-8
seq_len = 100_000 # sequence length
replicates = 100 # nubmer of replicates to simulate per model (there are 5000 per model in the dataset we will load.)
sample_size = 10 # number of individuals to sample per extant population


## Simulate data

Now, we are ready to simulate data under our models. We will summarize our data using the Site Frequency Spectrum (SFS).

In [4]:
def simulate_data(tdiv_prior, ne_prior, sample_size, seq_len, mu, rec_rate, replicates):

    # lists for storing matrices, responses
    all_sfs = []
    all_responses = []

    
    for model in ['divergence', 'geneflow']:
        
        for i in range(replicates):

            # draw parameters from priors
            this_tdiv = random.randint(tdiv_prior[0], tdiv_prior[1])
            this_ne_anc = random.randint(ne_prior[0], ne_prior[1])
            this_ne_1 = random.randint(ne_prior[0], ne_prior[1])
            this_ne_2 = random.randint(ne_prior[0], ne_prior[1])
            if model == "geneflow":
                this_migrate = np.random.uniform(low=migrate_prior[0], high=migrate_prior[1])
               
            # get our demography (using the function defined above)  
            if model == "divergence":
                demography = divergence(tdiv=this_tdiv, ne_anc=this_ne_anc, ne_1=this_ne_1, ne_2=this_ne_2)
            elif model == "geneflow":
                demography = geneflow(tdiv=this_tdiv, migrate=this_migrate, ne_anc=this_ne_anc, 
                                    ne_1=this_ne_1, ne_2=this_ne_2)

            # simulate tree sequences
            ts = msprime.sim_ancestry(samples={"A": sample_size, "B": sample_size}, 
                            demography = demography, sequence_length=seq_len, recombination_rate = rec_rate)
    
            # simulate mutations
            mts = msprime.sim_mutations(ts, rate=mu)

            # SFS
            sfs = mts.allele_frequency_spectrum(sample_sets=[mts.samples(0), mts.samples(1)], span_normalise=False, polarised=True)
    
            # get responses
            if model == 'divergence':
                response = 0
            elif model == 'geneflow':
                response = 1
    
            # append to lists
            all_sfs.append(sfs)
            all_responses.append(response)
    
    # convert lists to numpy arrays
    all_sfs = np.stack(all_sfs, axis=0)
    all_sfs = np.expand_dims(all_sfs, axis=-1)
    all_responses = np.stack(all_responses)
    
    
    return(all_sfs, all_responses)


simulated_sfs, simulated_responses = simulate_data(tdiv_prior, ne_prior, sample_size, seq_len, mu, rec_rate, replicates)

# Save or Load
To save some time, we will load a numpy array with more simulations

In [5]:
simulated_sfs = np.load("simulated_sfs.npy")
simulated_responses = np.load("simulated_responses.npy")

# Train a Machine Learning classifier!

Now, we have data we can use to train our classifier. We need to build and train our classifier!

## Build the classifier

In [6]:
# create our layers
conv1 = layers.Conv2D(10, (3, 3), activation='relu', input_shape=simulated_sfs.shape[1:])
pool1 = layers.MaxPooling2D((2, 2))
conv2 = layers.Conv2D(10, (3, 3), activation='relu')
pool2 = layers.MaxPooling2D((2, 2))
flatten = layers.Flatten(name="flatten")
dense50 = layers.Dense(50, activation='relu')
dense20 = layers.Dense(20, activation='relu')
densefinal = layers.Dense(2, activation='softmax', name="final")

# specify input
x = layers.Input(shape=simulated_sfs.shape[1:])

# build our model
x1 = conv1(x)
x1 = pool1(x1)
x1 = conv2(x1)
x1 = pool2(x1)
x1 = flatten(x1)
x1 = dense50(x1)
x1 = dense20(x1)
outputs = densefinal(x1)

# compile the model
model = models.Model(inputs=x, outputs=outputs)    
model.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss="categorical_crossentropy", metrics='accuracy')

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 21, 21, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 9, 9, 10)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 10)          910       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 10)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 90)                0     

2024-05-06 20:37:47.499125: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Train the model

In [7]:
# divide data into training, test, and validation
train_features, test_val_features, train_labels, test_val_labels = train_test_split(simulated_sfs, simulated_responses, test_size = 0.4)
test_features, val_features, test_labels, val_labels = train_test_split(test_val_features, test_val_labels, test_size = 0.2)

# convert labels to categorical 
train_labels_cat = to_categorical(train_labels, num_classes=2)
test_labels_cat = to_categorical(test_labels, num_classes=2)
val_labels_cat = to_categorical(val_labels, num_classes=2)

# fit our model
history = model.fit(train_features, train_labels_cat, epochs=10, 
                        validation_data=(val_features, val_labels_cat), batch_size=20)

Epoch 1/10
300/300 [==============================] - 2s 5ms/step - loss: 2.3508 - accuracy: 0.4900 - val_loss: 1.2233 - val_accuracy: 0.5400
Epoch 2/10
300/300 [==============================] - 1s 4ms/step - loss: 0.7773 - accuracy: 0.6602 - val_loss: 0.4583 - val_accuracy: 0.7987
Epoch 3/10
300/300 [==============================] - 1s 4ms/step - loss: 0.3491 - accuracy: 0.8597 - val_loss: 0.2420 - val_accuracy: 0.9137
Epoch 4/10
300/300 [==============================] - 1s 4ms/step - loss: 0.2065 - accuracy: 0.9295 - val_loss: 0.1610 - val_accuracy: 0.9525
Epoch 5/10
300/300 [==============================] - 1s 4ms/step - loss: 0.1462 - accuracy: 0.9525 - val_loss: 0.1244 - val_accuracy: 0.9600
Epoch 6/10
300/300 [==============================] - 1s 4ms/step - loss: 0.1154 - accuracy: 0.9635 - val_loss: 0.1043 - val_accuracy: 0.9675
Epoch 7/10
300/300 [==============================] - 1s 4ms/step - loss: 0.0971 - accuracy: 0.9690 - val_loss: 0.0904 - val_accuracy: 0.9712
Epoch 

## Test the model

It is important to test the model on an indpendent dataset. We did not use our test data for validation (which would be important if we adjusted model parameters to improve performance), or for training.

In [8]:
predict = model.predict(test_features)
predicted_labels = np.argmax(predict, axis=1)
matrix = confusion_matrix(test_labels, predicted_labels)
accuracy = accuracy_score(test_labels, predicted_labels)
print(matrix)
print(accuracy)

100/100 [==============================] - 0s 2ms/step
[[1593   29]
 [  52 1526]]
0.9746875


# References

Baumdicker F, et al. (2022), Efficient ancestry and mutation simulation with msprime 1.0, *Genetics*, 220(3): iyab229. 
http://doi.org/10.1371/journal.pcbi.1004842

Haller BC, Messer PW. (2023) SLiM 4: Multispecies eco-evolutionary modeling, *The American Naturalist*, 201(5): In press. https://doi.org/10.1086/723601

Adrion JR, et al. (2020), A community-maintained standard library of population genetic models, *eLife* 9:e54967, https://doi.org/10.7554/eLife.54967

